In [22]:
import pandas as pd

# Reading The Excel File


In [29]:
#reading data
data=pd.read_excel('115_PastPerformance-2-PYTHON-INPUT.xlsx')

In [30]:
data

newRepoDate   RecoPr  ExitPrice  newExitDate    
0      20220407   124.50     135.35     20220419 NaN
1      20220414   433.15     494.25     20220418 NaN
2      20220421   419.80     472.50     20220504 NaN
3      20220428  1418.00    1582.70     20220517 NaN
4      20220505   308.90     337.20     20220725 NaN
..          ...      ...        ...          ...  ..
95     20240307   592.05        NaN           -1 NaN
96     20240314   384.45        NaN           -1 NaN
97     20240321   588.50     666.85     20240402 NaN
98     20240328   263.10     291.15     20240402 NaN
99     20240404   536.85        NaN           -1 NaN

[100 rows x 5 columns]

In [31]:
recordprice=list(data.RecoPr)
exitprice=list(data.ExitPrice)
newRecoDate=list(data.newRepoDate)
newExitDate=list(data.newExitDate)

# main code for calculating the minimum amount for investment

In [41]:
import heapq

# IMPORTANT NOTES

# WE ARE USING THE DATE IN FORMAT YYMMDDDD, as it's easire the sort and compare directly
# like for Columns data in'newExitDate' && 'newRecoDate'


# 1 Lakh == 100 Thousand
# 1 Lakh == 100,000.00


class PriorityQueue:
    def __init__(self):
        self.heap = []

    def push(self, priority, item):
        heapq.heappush(self.heap, (priority, item))

    def pop(self):
        if self.is_empty():
            raise IndexError("pop from an empty priority queue")
        _, item = heapq.heappop(self.heap)
        return item

    def is_empty(self):
        return len(self.heap) == 0
    
    def front(self):
        if self.is_empty():
            raise IndexError("front from an empty priority queue")
        return self.heap[0][0]
    def back(self):
        if self.is_empty():
            raise IndexError("back from an empty priority queue")
        return self.heap[0][1]
    
    
    

# using priority queue
priority_queue = PriorityQueue()
funds = []

# units brougth for each stock
units = []

# amountused for each stock (we assume that we have nearly invested 100,000.00 or 1lakh on each stock)
amountused = [] 

valuereturn = []


# returnamount is the amount we got when we sold the stock on its exit date
returnamount = [] 

# moneyinvested will only store the amount we have used from our pockets, that is the pure investment from us
# we don't inclued the money we have invested in a stock using the returns of another stock
moneyinvested = 0

# remaining amount after each stock purchasing
remain=0

# corpus_amount is the money which has been invested into the market but their exit date hasn't been declared yet
# thus this after the investing in the final stock, the funds which will be stuck in the market are placed here
corpus_amount = 0


# the range 0-100 is a manual input, thus describing the number of rows to be read from the file
# which can vary from file to file, so please check the number of rows before running the cell
# as for this example, we know we have 101 to read, but the 1st row is of heading thus ignoring it,
# we have 100 rows
row_size = 100


# again saying this, the number 99 is the last row the input file,
# it may vary from
last_row = 99


for i in range(0,row_size):
    # checking if the ExitDate doesn't exists
    
    if(newExitDate[i] == -1): 
        #print(remain)
        
        if(remain>=100000): # if we have more then 1lakh of money in returns
            remain=remain-100000 # thus don't add money from our pocket use this money 
            funds.append(100000) # add it to the funds
            
        # check if the Reco Date for a stock is more then the Exit Date for the front stock
        elif(newRecoDate[i]>priority_queue.front()): 
            # checking if the return amount for that stock is more than 100000, or 1 lakh
            if(priority_queue.back()>=100000):
                
                # check if we have more then 1lakh of money in returns
                if(remain>=100000):
                    remain=remain-100000 # don't add money from our pocket use this money
                
                remain=remain+(priority_queue.pop()-100000) # increase the amount of 'remain' that is we got the return of that stock,
                                                            # which is more than 1 lakh
                funds.append(100000) # add it to the funds
                
                
            # if the return amount is less than 1 lakh, this also tells us that we had a loss,
            # we have have invested approximately 1lakh in each stock
            else: 
                remain=remain+priority_queue.pop() #just add the returned amount to the remains
                
                # check if we have more then 1lakh of money in returns
                if(remain>=100000): 
                    remain=remain-100000 # don't add money from our pocket use this money
                    funds.append(100000) # add it to the funds
                    
                #else we invested 1 lakh money from our pocket, and assume that the 'remain' amount will be used in future
                else:
                    moneyinvested+=100000
                    funds.append(100000)
                    
        # if any of the conditions is not true, that is we dont have 'remain' amount more than 1 lakh and
        # we Dont have a stock who Reco Date is greater (or coming before) any Exit Date (which in the front the priority queue)
        else:
            # check if we have more then 1lakh of money in returns
            if(remain>=100000):
                remain=remain-100000 # don't add money from our pocket use this money
                funds.append(100000) # add it to the funds
            else:
                moneyinvested=moneyinvested+100000 # here we are putting money from our pocket
                funds.append(100000) 
                
                
            
        units.append(int(funds[i]/recordprice[i])) # number of units we purchased for that stock
        remain=remain+(funds[i]-(units[i]*recordprice[i])) # remaining amount
        amountused.append(round((units[i]*recordprice[i]),2)) # amount used to purchase that stock
        returnamount.append(0);
        valuereturn.append(round(units[i]*recordprice[i],2)) 
        corpus_amount += round(units[i]*recordprice[i],2) # amount which is invested but will not be retunred as not Exit Date has been set

        continue
        
    # check if priority queue is empty and the stock that is being brought has a Exit Date, 
    # it also indicates that right now we dont any stock who's return we will get
    
    elif priority_queue.is_empty(): 
        funds.append(100000)  # add it to the funds
        moneyinvested=moneyinvested+100000 # here we are putting money from our pocket
        units.append(int(funds[i]/recordprice[i])) # number of units we purchased for that stock
        remain=remain+funds[i]-(units[i]*recordprice[i]) # remaining amount
        amountused.append(round((units[i]*recordprice[i]),2))
        valuereturn.append(round(units[i]*exitprice[i],2))
        returnamount.append(round((units[i]*exitprice[i]-units[i]*recordprice[i]),2))
        priority_queue.push(int(newExitDate[i]),units[i]*exitprice[i])
        
        
    # if none the above conditions are true, that is we have a stock with an Exit Date
    # and the priority queue is not empty
    else:
        # check if we have more then 1lakh of money in returns
        if(remain>=100000):
            remain=remain-100000 # don't add money from our pocket use this money
            funds.append(100000) # add it to the funds
            
            
        # check if the Reco Date for a stock is more then the Exit Date for the front stock
        elif(newRecoDate[i]>priority_queue.front()):
            # checking if the return amount for that stock is more than 100000, or 1 lakh
            if(priority_queue.back()>=100000):
                # check if we have more then 1lakh of money in returns
                if(remain>=100000):
                    remain=remain-100000
                
                remain=remain+(priority_queue.pop()-100000) # increase the amount of 'remain' that is we got the return of that stock,
                                                            # which is more than 1 lakh
                funds.append(100000) # add it to the funds
                
            # if the return amount is less than 1 lakh, this also tells us that we had a loss,
            # we have have invested approximately 1lakh in each stock
            else:
                remain=remain+priority_queue.pop()
    
                # check if we have more then 1lakh of money in returns
                if(remain>=100000):
                    remain=remain-100000 # don't add money from our pocket use this money
                    funds.append(100000) # add it to the funds
                else:
                    moneyinvested+=100000 # here we are putting money from our pocket
                    funds.append(100000) # add it to the funds
        else:
            funds.append(100000)
            moneyinvested=moneyinvested+100000 # here we are putting money from our pocket
            
        
        units.append(int(funds[i]/recordprice[i])) # number of units we purchased for that stock
        remain=remain+(funds[i]-(units[i]*recordprice[i]))
        #print(remain)
        amountused.append(round((units[i]*recordprice[i]),2))
        valuereturn.append(round(units[i]*exitprice[i],2))
        returnamount.append(round((units[i]*exitprice[i]-units[i]*recordprice[i]),2))
        priority_queue.push(int(newExitDate[i]),units[i]*exitprice[i])

val=priority_queue.front()
#print(priority_queue.front())

fund_not_used = 0
futuredatemoney=0



# using the last Reco Date for our reference use
lastRecoDate = int(newRecoDate[last_row])

#now after investing in all the stock that is the last Repo Date, and getting the retuns for all those stock
#there will be still stock invested who's exit are yet to come, thus we are computing for all that amount
while(priority_queue.is_empty() == 0):
    print(priority_queue.front(), " ",priority_queue.back())
    temp=priority_queue.front()
    if(lastRecoDate - temp < 0):
        futuredatemoney += priority_queue.back()
    else:
        fund_not_used += priority_queue.back()
    priority_queue.pop()

    
    


20240212   111840.3
20240220   113954.79999999999
20240223   107423.75
20240226   111940.40000000001
20240227   83619.0
20240402   110636.99999999999
20240402   110970.0
20240402   112697.65000000001


##### 

# Seeing all the Values

In [33]:
print("Amount used")
print(amountused)
print(len(amountused))
#print(sum(amountused))


print("Amount returned")
print(valuereturn)
print(len(valuereturn))
#print(sum(valuereturn))

print()
print("Returnamount")
print(returnamount)
print(len(returnamount))

print("Units")
print(units)
print(len(units))

print()
print("Remaining amount")
print(remain)


print()
print("Money Invested")
print(moneyinvested)

    

Amount used
[99973.5, 99624.5, 99912.4, 99260.0, 99774.7, 99718.3, 99892.0, 94901.3, 99875.2, 99918.0, 99990.0, 98706.8, 98318.0, 99918.0, 99647.0, 99897.6, 99661.8, 99935.25, 99596.0, 99366.75, 99943.2, 99900.0, 99012.0, 99866.0, 99002.4, 99829.8, 99883.55, 99935.0, 99942.75, 99853.95, 99844.8, 99961.05, 99766.35, 99559.6, 99200.0, 99707.2, 99813.5, 98241.0, 99416.0, 99480.0, 99770.4, 98781.4, 99858.5, 99281.0, 99669.2, 99793.35, 99577.8, 99899.0, 99723.0, 99278.4, 99484.0, 99900.2, 99696.0, 99642.15, 99589.9, 99952.65, 99837.45, 99855.15, 99430.0, 99913.7, 99603.8, 99500.4, 99957.42, 99500.0, 99695.25, 99887.5, 99840.0, 99400.0, 99844.5, 99959.75, 99430.8, 99716.1, 99684.5, 99846.8, 99950.5, 99359.8, 99105.95, 99653.8, 99951.6, 99900.6, 99048.95, 97714.75, 97032.0, 99567.6, 99667.2, 99993.6, 99299.7, 99724.8, 99975.15, 99897.3, 99931.05, 99963.76, 99563.1, 99934.65, 99900.0, 99464.4, 99957.0, 99456.5, 99978.0, 99854.1]
100
Amount returned
[108686.05, 113677.5, 112455.0, 110789.0, 108

# Displaying the All the Money Used, how and where

In [34]:
print("FINAL Corpus Amount : ie the money which is invested in the market but not returned yet :")
print(corpus_amount)
print()
print("Amount of money not used after last Reco date")
print(fund_not_used)
print()

print("Minimum Invested Money : ie the only money which we had used from our pocket thus not using the profit from other returns")
print(moneyinvested)
print()

# print("Money which will come after last reco date:")
# print(futuredatemoney)
# print("Amount used without any return")
print("Total amount invested :")
print(sum(amountused))
print()

print("Total amount returned :")
print(sum(valuereturn))
print()

print("Total profit : Total invested - Total returned")
print(sum(valuereturn)-sum(amountused))
print()
print("Profit % of the above Total Profit")
print((sum(valuereturn)-sum(amountused))*100/sum(amountused))

print()

# print("profit")
# print(((corpus_amount + fund_not_used+remain) -(moneyinvested)))
# print()

print("Actual Profit % : the profit only from our invested money ")
print(((corpus_amount + fund_not_used+remain) -(moneyinvested))*100/moneyinvested)

FINAL Corpus Amount : ie the money which is invested in the market but not returned yet :
997504.0

Amount of money not used after last Reco date
863082.9

Minimum Invested Money : ie the only money which we had used from our pocket thus not using the profit from other returns
1000000

Total amount invested :
9956691.18

Total amount returned :
10833232.620000001

Total profit : Total invested - Total returned
876541.4400000013

Profit % of the above Total Profit
8.803541499416088

Actual Profit % : the profit only from our invested money 
87.65414399999997
